# Clinical NLP System for Neonatal Discharge Summary Entity Extraction

This notebook demonstrates a comprehensive clinical NLP system designed to extract entities from neonatal discharge summaries.

## Target Entities
- **P_ID**: Patient identifier
- **Gestational_Age**: Age at birth in weeks
- **Sex**: Gender (Male/Female)
- **Birth_Weight**: Weight at birth
- **Diagnosis**: Medical diagnosis
- **Treatment_Respiratory**: Respiratory treatment information
- **Treatment_Medication**: Medication treatment details
- **Outcome**: Patient outcome/discharge status

## Output Formats
1. JSON structured format with each entity as a key-value pair
2. Markdown table format with one row
3. Evaluation metrics: Precision, Recall, and F1-score

## 1. Import Required Libraries

Import essential libraries for NLP processing, evaluation, and visualization.

In [ ]:
import sys
import os

# Add the parent directory to the path to import our modules
sys.path.append(os.path.dirname(os.getcwd()))

# Core libraries
import pandas as pd
import numpy as np
import json
import re
from typing import Dict, List, Optional

# NLP libraries
try:
    import spacy
    print("spaCy loaded successfully")
except ImportError:
    print("spaCy not installed. Install with: pip install spacy")

# Evaluation libraries
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Our custom modules
try:
    from neonatal_ner import NeonatalNER
    from entity_patterns import EntityPatterns
    from evaluation import EntityEvaluator
    print("Custom modules loaded successfully")
except ImportError as e:
    print(f"Could not import custom modules: {e}")
    print("Make sure you're running this from the notebooks directory")

print("All libraries imported successfully!")

## 2. Load and Preprocess Sample Data

Load sample neonatal discharge summary sentences and their ground truth annotations.

In [ ]:
# Load sample annotations
with open('../data/sample_annotations.json', 'r') as f:
    sample_data = json.load(f)

# Extract texts and annotations
sample_texts = [item['text'] for item in sample_data]
sample_annotations = [item['annotations'] for item in sample_data]

print(f"Loaded {len(sample_texts)} sample texts")
print("\nFirst sample:")
print(f"Text: {sample_texts[0]}")
print(f"Annotations: {json.dumps(sample_annotations[0], indent=2)}")

# Create a DataFrame for easier viewing
df_samples = pd.DataFrame(sample_data)
print("\nSample data overview:")
print(df_samples.head())

## 3. Define Entity Extraction Classes

Initialize our clinical patterns and NER system.

In [ ]:
# Initialize the entity patterns
patterns = EntityPatterns()

# Display some example patterns
print("Sample Patient ID Patterns:")
for i, pattern in enumerate(patterns.patient_id_patterns[:3]):
    print(f"{i+1}. {pattern}")

print("\nSample Gestational Age Patterns:")
for i, pattern in enumerate(patterns.gestational_age_patterns[:3]):
    print(f"{i+1}. {pattern}")

print("\nSample Sex Patterns:")
for i, pattern in enumerate(patterns.sex_patterns[:3]):
    print(f"{i+1}. {pattern}")

print("\nAll pattern categories:")
all_patterns = patterns.get_all_patterns()
for category, pattern_list in all_patterns.items():
    print(f"- {category}: {len(pattern_list)} patterns")

## 4. Build Rule-Based Entity Extractor

Initialize our NeonatalNER system and test it on sample text.

In [ ]:
# Initialize the NER system
ner_system = NeonatalNER()

# Test on the first sample
test_text = sample_texts[0]
test_ground_truth = sample_annotations[0]

print("Testing on sample text:")
print(f"Text: {test_text}")
print("\nGround Truth:")
print(json.dumps(test_ground_truth, indent=2))

# Extract entities
result = ner_system.extract_entities(test_text, test_ground_truth)

print("\nExtracted Entities:")
print(json.dumps(result['entities'], indent=2))

## 5. Implement spaCy NER Model (Rule-based approach)

For demonstration, we'll use our rule-based approach which is more suitable for clinical text patterns.

In [ ]:
# Test individual entity extraction methods
print("Testing individual entity extraction:")
print("="*50)

test_text = sample_texts[0]
print(f"Text: {test_text}")
print()

# Test each entity extraction
entities = ner_system._extract_all_entities(test_text)

entity_names = [
    'P_ID', 'Gestational_Age', 'Sex', 'Birth_Weight',
    'Diagnosis', 'Treatment_Respiratory', 'Treatment_Medication', 'Outcome'
]

for entity_name in entity_names:
    extracted_value = getattr(entities, entity_name)
    ground_truth_value = test_ground_truth.get(entity_name)
    
    status = "✓" if extracted_value == ground_truth_value else "✗"
    
    print(f"{entity_name:20}: {extracted_value or 'null':30} | GT: {ground_truth_value or 'null':30} {status}")

## 6. Create Output Formatting Functions

Demonstrate JSON and Markdown table formatting.

In [ ]:
# Extract entities and format outputs
result = ner_system.extract_entities(test_text, test_ground_truth)

print("1. JSON FORMAT:")
print("="*50)
print(result['json_output'])

print("\n\n2. MARKDOWN TABLE FORMAT:")
print("="*50)
print(result['markdown_table'])

# Display the markdown table in a more readable format
print("\n\n3. FORMATTED MARKDOWN TABLE:")
print("="*50)
from IPython.display import Markdown, display
display(Markdown(result['markdown_table']))

## 7. Develop Evaluation Metrics Calculator

Calculate and display evaluation metrics.

In [ ]:
# Display evaluation metrics
if 'evaluation' in result:
    eval_data = result['evaluation']
    
    print("EVALUATION METRICS:")
    print("="*50)
    print(f"Precision: {eval_data['precision']:.3f} ({eval_data['correct_extractions']}/{eval_data['total_extractions']})")
    print(f"Recall:    {eval_data['recall']:.3f} ({eval_data['correct_extractions']}/{eval_data['total_actual']})")
    print(f"F1-Score:  {eval_data['f1_score']:.3f}")
    
    print("\nPER-ENTITY RESULTS:")
    print("-" * 50)
    for entity, status in eval_data['entity_results'].items():
        print(f"{entity:20}: {status}")
        
    # Create a simple visualization
    fig, ax = plt.subplots(1, 1, figsize=(8, 5))
    
    metrics = ['Precision', 'Recall', 'F1-Score']
    values = [eval_data['precision'], eval_data['recall'], eval_data['f1_score']]
    
    bars = ax.bar(metrics, values, color=['skyblue', 'lightgreen', 'gold'])
    ax.set_ylim(0, 1)
    ax.set_ylabel('Score')
    ax.set_title('Evaluation Metrics for Single Sample')
    
    # Add value labels on bars
    for bar, value in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                f'{value:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
else:
    print("No evaluation data available")

## 8. Train Custom NER Model (Batch Processing)

Process all samples and evaluate overall performance.

In [ ]:
# Process all samples
all_results = ner_system.batch_extract(sample_texts, sample_annotations)

print(f"Processed {len(all_results)} samples")
print("\nSample Results:")
print("="*80)

for i, result in enumerate(all_results[:3]):  # Show first 3 results
    print(f"\nSample {i+1}:")
    print(f"Text: {sample_texts[i][:100]}...")
    
    if 'evaluation' in result:
        eval_data = result['evaluation']
        print(f"Precision: {eval_data['precision']:.3f}, Recall: {eval_data['recall']:.3f}, F1: {eval_data['f1_score']:.3f}")
    
    print("Extracted entities:")
    for entity, value in result['entities'].items():
        print(f"  {entity}: {value or 'null'}")
        
print("\n" + "="*80)

## 9. Test Entity Extraction Pipeline

Comprehensive pipeline demonstration with a new test case.

In [ ]:
# Create a new test case
new_test_text = """
Patient ID: NB-2024-007. Female infant born at 31 weeks gestational age with birth weight of 1650 grams.
Primary diagnosis: Severe respiratory distress syndrome and patent ductus arteriosus.
Treatment included mechanical ventilation for 5 days followed by CPAP support.
Medications: Surfactant therapy, caffeine citrate for apnea, and indomethacin for PDA closure.
Patient showed good improvement and was discharged home after 6 weeks in stable condition.
"""

new_ground_truth = {
    "P_ID": "NB-2024-007",
    "Gestational_Age": "31 weeks",
    "Sex": "Female",
    "Birth_Weight": "1650 grams",
    "Diagnosis": "Severe respiratory distress syndrome and patent ductus arteriosus",
    "Treatment_Respiratory": "mechanical ventilation for 5 days followed by CPAP support",
    "Treatment_Medication": "Surfactant therapy, caffeine citrate, indomethacin",
    "Outcome": "discharged home after 6 weeks in stable condition"
}

print("COMPLETE PIPELINE TEST")
print("="*80)
print(f"Input Text: {new_test_text.strip()}")
print("\nGround Truth:")
print(json.dumps(new_ground_truth, indent=2))

# Run the complete pipeline
pipeline_result = ner_system.extract_entities(new_test_text, new_ground_truth)

# Display results using the built-in pretty printer
ner_system.print_results(pipeline_result)

## 10. Performance Analysis and Visualization

Analyze performance across different entity types and create comprehensive visualizations.

In [ ]:
# Collect evaluation results from all samples
evaluator = EntityEvaluator()
all_predictions = [result['entities'] for result in all_results]
batch_evaluation = evaluator.batch_evaluate(all_predictions, sample_annotations)

print("BATCH EVALUATION RESULTS")
print("="*80)
print(f"Overall Precision: {batch_evaluation['aggregate_precision']:.3f}")
print(f"Overall Recall: {batch_evaluation['aggregate_recall']:.3f}")
print(f"Overall F1-Score: {batch_evaluation['aggregate_f1_score']:.3f}")
print(f"Total Samples: {batch_evaluation['sample_count']}")

# Per-entity performance
per_entity = batch_evaluation['per_entity_metrics']
print("\nPER-ENTITY PERFORMANCE:")
print("-" * 80)

entity_data = []
for entity, metrics in per_entity.items():
    entity_data.append({
        'Entity': entity,
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score'],
        'Correct': metrics['correct'],
        'Total_Predicted': metrics['total_predicted'],
        'Total_Actual': metrics['total_actual']
    })
    print(f"{entity:20}: P={metrics['precision']:.3f}, R={metrics['recall']:.3f}, F1={metrics['f1_score']:.3f}")

# Create DataFrame for visualization
df_performance = pd.DataFrame(entity_data)
print("\nPerformance DataFrame:")
print(df_performance.round(3))

In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Overall Metrics Bar Chart
overall_metrics = ['Precision', 'Recall', 'F1-Score']
overall_values = [
    batch_evaluation['aggregate_precision'],
    batch_evaluation['aggregate_recall'],
    batch_evaluation['aggregate_f1_score']
]

bars1 = axes[0, 0].bar(overall_metrics, overall_values, color=['skyblue', 'lightgreen', 'gold'])
axes[0, 0].set_ylim(0, 1)
axes[0, 0].set_title('Overall Performance Metrics')
axes[0, 0].set_ylabel('Score')

for bar, value in zip(bars1, overall_values):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                   f'{value:.3f}', ha='center', va='bottom')

# 2. Per-Entity F1 Scores
entities = df_performance['Entity']
f1_scores = df_performance['F1-Score']

bars2 = axes[0, 1].bar(range(len(entities)), f1_scores, color='coral')
axes[0, 1].set_xticks(range(len(entities)))
axes[0, 1].set_xticklabels(entities, rotation=45, ha='right')
axes[0, 1].set_ylim(0, 1)
axes[0, 1].set_title('F1-Score by Entity Type')
axes[0, 1].set_ylabel('F1-Score')

for i, (bar, value) in enumerate(zip(bars2, f1_scores)):
    axes[0, 1].text(i, value + 0.01, f'{value:.2f}', ha='center', va='bottom', fontsize=8)

# 3. Precision vs Recall Scatter Plot
precision_vals = df_performance['Precision']
recall_vals = df_performance['Recall']

scatter = axes[1, 0].scatter(precision_vals, recall_vals, c=f1_scores, cmap='viridis', s=100, alpha=0.7)
axes[1, 0].set_xlabel('Precision')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].set_title('Precision vs Recall by Entity')
axes[1, 0].set_xlim(0, 1)
axes[1, 0].set_ylim(0, 1)
axes[1, 0].grid(True, alpha=0.3)

# Add entity labels to scatter plot
for i, entity in enumerate(entities):
    axes[1, 0].annotate(entity, (precision_vals.iloc[i], recall_vals.iloc[i]), 
                       xytext=(5, 5), textcoords='offset points', fontsize=8)

# Add colorbar
plt.colorbar(scatter, ax=axes[1, 0], label='F1-Score')

# 4. Entity Extraction Success Rate
success_rates = df_performance['Correct'] / df_performance['Total_Actual'] * 100
success_rates = success_rates.fillna(0)  # Handle division by zero

bars4 = axes[1, 1].bar(range(len(entities)), success_rates, color='lightsteelblue')
axes[1, 1].set_xticks(range(len(entities)))
axes[1, 1].set_xticklabels(entities, rotation=45, ha='right')
axes[1, 1].set_ylim(0, 100)
axes[1, 1].set_title('Extraction Success Rate by Entity')
axes[1, 1].set_ylabel('Success Rate (%)')

for i, (bar, value) in enumerate(zip(bars4, success_rates)):
    axes[1, 1].text(i, value + 1, f'{value:.0f}%', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSUMMARY STATISTICS:")
print("="*50)
print(f"Best performing entity (F1): {entities[f1_scores.idxmax()]} ({f1_scores.max():.3f})")
print(f"Worst performing entity (F1): {entities[f1_scores.idxmin()]} ({f1_scores.min():.3f})")
print(f"Average F1-Score: {f1_scores.mean():.3f}")
print(f"Standard deviation: {f1_scores.std():.3f}")

## Conclusion

This notebook demonstrates a comprehensive clinical NLP system for extracting entities from neonatal discharge summaries. The system:

1. **Identifies 8 key entities** relevant to neonatal care
2. **Provides dual output formats** (JSON and Markdown table)
3. **Includes evaluation metrics** (Precision, Recall, F1-score)
4. **Uses clinical pattern matching** optimized for medical text
5. **Provides detailed performance analysis** across entity types

### Key Features:
- Rule-based approach suitable for clinical terminology
- Comprehensive evaluation framework
- Batch processing capabilities
- Detailed performance visualization
- Easily extensible pattern system

### Next Steps:
1. Expand training data with more annotated samples
2. Implement transformer-based models for comparison
3. Add more sophisticated entity linking
4. Integrate with clinical workflow systems
5. Implement active learning for continuous improvement